# 0. Imports

In [9]:
import pandas as pd
import numpy as np
import re

import sys 
sys.path.append("..")
from src.data_cleaning_support import convert_to_hours

import ast

# 1. Cleaning of travel tables

# 1.1 Flights

In [10]:
flight_itinieraries = pd.read_csv("../data/flights/flight_itinieraries_weeks_destinations.csv", index_col=0)

In [11]:
flight_itinieraries.info()
flight_itinieraries.head()

<class 'pandas.core.frame.DataFrame'>
Index: 8144 entries, 0 to 57
Data columns (total 23 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   score                          8144 non-null   float64
 1   price                          8144 non-null   int64  
 2   price_currency                 8144 non-null   object 
 3   duration_departure             8144 non-null   int64  
 4   duration_return                8144 non-null   int64  
 5   stops_departure                8144 non-null   int64  
 6   stops_return                   8144 non-null   int64  
 7   departure_departure            8144 non-null   object 
 8   arrival_departure              8144 non-null   object 
 9   departure_return               8144 non-null   object 
 10  arrival_return                 8144 non-null   object 
 11  company_departure              8144 non-null   object 
 12  company_return                 8144 non-null   object 


,score,price,price_currency,duration_departure,duration_return,stops_departure,stops_return,departure_departure,arrival_departure,departure_return,...,self_transfer,fare_isChangeAllowed,fare_isPartiallyChangeable,fare_isCancellationAllowed,fare_isPartiallyRefundable,origin_airport_departure,destination_airport_departure,origin_airport_return,destination_airport_return,week
0,0.999000,248,€,135,135,0,0,2024-11-04 14:25:00,2024-11-04 16:40:00,2024-11-11 11:35:00,...,False,False,False,False,False,Madrid,Paris Charles de Gaulle,Paris Charles de Gaulle,Madrid,1
1,0.902639,187,€,130,130,0,0,2024-11-04 20:00:00,2024-11-04 22:10:00,2024-11-11 17:15:00,...,False,False,False,False,False,Madrid,Paris Beauvais,Paris Beauvais,Madrid,1
2,0.821090,289,€,130,130,0,0,2024-11-04 16:20:00,2024-11-04 18:30:00,2024-11-11 07:00:00,...,False,False,False,False,False,Madrid,Paris Charles de Gaulle,Paris Charles de Gaulle,Madrid,1
3,0.774970,288,€,130,130,0,0,2024-11-04 17:40:00,2024-11-04 19:50:00,2024-11-11 14:45:00,...,False,False,False,False,False,Madrid,Paris Orly,Paris Orly,Madrid,1
4,0.759652,190,€,130,130,0,0,2024-11-04 20:00:00,2024-11-04 22:10:00,2024-11-11 06:40:00,...,False,False,False,False,False,Madrid,Paris Beauvais,Paris Beauvais,Madrid,1


In [12]:
flight_itinieraries["destination_city"] = flight_itinieraries["destination_airport_departure"].str.split(expand=True)[0]
flight_itinieraries

,score,price,price_currency,duration_departure,duration_return,stops_departure,stops_return,departure_departure,arrival_departure,departure_return,...,fare_isChangeAllowed,fare_isPartiallyChangeable,fare_isCancellationAllowed,fare_isPartiallyRefundable,origin_airport_departure,destination_airport_departure,origin_airport_return,destination_airport_return,week,destination_city
0,0.999000,248,€,135,135,0,0,2024-11-04 14:25:00,2024-11-04 16:40:00,2024-11-11 11:35:00,...,False,False,False,False,Madrid,Paris Charles de Gaulle,Paris Charles de Gaulle,Madrid,1,Paris
1,0.902639,187,€,130,130,0,0,2024-11-04 20:00:00,2024-11-04 22:10:00,2024-11-11 17:15:00,...,False,False,False,False,Madrid,Paris Beauvais,Paris Beauvais,Madrid,1,Paris
2,0.821090,289,€,130,130,0,0,2024-11-04 16:20:00,2024-11-04 18:30:00,2024-11-11 07:00:00,...,False,False,False,False,Madrid,Paris Charles de Gaulle,Paris Charles de Gaulle,Madrid,1,Paris
3,0.774970,288,€,130,130,0,0,2024-11-04 17:40:00,2024-11-04 19:50:00,2024-11-11 14:45:00,...,False,False,False,False,Madrid,Paris Orly,Paris Orly,Madrid,1,Paris
4,0.759652,190,€,130,130,0,0,2024-11-04 20:00:00,2024-11-04 22:10:00,2024-11-11 06:40:00,...,False,False,False,False,Madrid,Paris Beauvais,Paris Beauvais,Madrid,1,Paris
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53,0.172074,1424,€,1275,1840,1,2,2024-11-18 23:45:00,2024-11-19 15:00:00,2024-11-25 16:15:00,...,False,False,False,False,Madrid,Havana,Havana,Madrid,3,Havana
54,0.156223,1602,€,1845,1205,2,2,2024-11-18 20:35:00,2024-11-19 21:20:00,2024-11-25 14:00:00,...,False,False,False,False,Madrid,Havana,Havana,Madrid,3,Havana
55,0.139322,1734,€,2075,1085,1,1,2024-11-18 10:25:00,2024-11-19 15:00:00,2024-11-25 16:15:00,...,False,False,False,False,Madrid,Havana,Havana,Madrid,3,Havana
56,0.131635,1949,€,1890,1085,2,1,2024-11-18 13:30:00,2024-11-19 15:00:00,2024-11-25 16:15:00,...,False,False,False,False,Madrid,Havana,Havana,Madrid,3,Havana


Taking a lot at both the info method and the head of the flights dataframes for both cities, the data types and values seem correct. So no additional cleaning appart from clearly adding the destination city is needed here.

In [13]:
flight_itinieraries.to_csv("../data/flights/flight_itinieraries_weeks_destinations_cleaned.csv")

## 1.2 Accommodations

In [14]:
accommodations = pd.read_csv("../data/accommodations/acommodations.csv", index_col=0)

In [15]:
accommodations.info()
accommodations.head()

<class 'pandas.core.frame.DataFrame'>
Index: 8486 entries, 0 to 49
Data columns (total 19 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   name                     8486 non-null   object 
 1   url                      8486 non-null   object 
 2   price_currency           8486 non-null   object 
 3   total_price_amount       8486 non-null   int64  
 4   distance_city_center_km  3933 non-null   float64
 5   score                    8248 non-null   float64
 6   n_comments               8248 non-null   float64
 7   close_to_metro           8486 non-null   object 
 8   sustainability_cert      8486 non-null   object 
 9   room_type                8486 non-null   object 
 10  double_bed               8486 non-null   object 
 11  single_bed               8486 non-null   object 
 12  free_cancellation        8486 non-null   object 
 13  breakfast_included       8486 non-null   object 
 14  pay_at_hotel             8486 n

,name,url,price_currency,total_price_amount,distance_city_center_km,score,n_comments,close_to_metro,sustainability_cert,room_type,double_bed,single_bed,free_cancellation,breakfast_included,pay_at_hotel,location_score,free_taxi,week,type
0,Hotel Meslay Republique,https://www.booking.com/hotel/fr/meslay-republ...,€,1143,1.4,8.1,3153.0,Yes,No,Habitación Doble Estándar - 2 camas,No,Yes,No,No,No,8.2,No,1,solitary
1,Superbe Triplex au Coeur de Montorgueil,https://www.booking.com/hotel/fr/superbe-tripl...,€,1272,1.1,8.5,28.0,Yes,No,Casa con 1 dormitorio,Yes,No,No,No,No,8.4,No,1,solitary
2,Moxy Paris Bastille,https://www.booking.com/hotel/fr/moxy-paris-ba...,€,1311,1.4,8.0,1337.0,Yes,No,MOXY Sleeper con cama grande,Yes,No,Yes,Yes,Yes,8.2,No,1,solitary
3,MBA Splendide Appart Saint-Michel Paris Centre,https://www.booking.com/hotel/fr/splendide-app...,€,776,0.8,7.1,15.0,Yes,No,Apartamento - Planta baja,No,No,No,No,No,NaN,No,1,solitary
4,Paris Bastille,https://www.booking.com/hotel/fr/hotel-paris-b...,€,1304,1.5,8.2,1919.0,Yes,No,Habitación Doble - 1 o 2 camas,Yes,Yes,No,No,No,8.6,No,1,solitary


Judging from both the info method and the head of the accommodation dataframes for both cities, the data types and values seem correct. So no cleaning is needed here. 

However, the destination city is lacking from the dataframe due to a mistake in its registration. In order to correct that, the url can be used, as it starts by different country prefix for every city:

In [16]:
accommodations["url"].str[0:33].unique()
choices = ['https://www.booking.com/hotel/fr/',
       'https://www.booking.com/hotel/de/',
       'https://www.booking.com/hotel/th/',
       'https://www.booking.com/hotel/it/',
       'https://www.booking.com/hotel/us/',
       'https://www.booking.com/hotel/pt/',
       'https://www.booking.com/hotel/es/']

Making use of numpy choices method to create the 'destination' column:

In [17]:
conditions = [
    accommodations['url'].str.startswith('https://www.booking.com/hotel/fr/'),
    accommodations['url'].str.startswith('https://www.booking.com/hotel/de/'),
    accommodations['url'].str.startswith('https://www.booking.com/hotel/th/'),
    accommodations['url'].str.startswith('https://www.booking.com/hotel/it/'),
    accommodations['url'].str.startswith('https://www.booking.com/hotel/us/'),
    accommodations['url'].str.startswith('https://www.booking.com/hotel/pt/'),
    accommodations['url'].str.startswith('https://www.booking.com/hotel/es/')
]

choices = ['Paris', 'Berlin', 'Phuket', 'Rome', 'New York', 'Madeira', 'Havana']

accommodations['destination'] = np.select(conditions, choices, default='Unknown')

In [18]:
accommodations

,name,url,price_currency,total_price_amount,distance_city_center_km,score,n_comments,close_to_metro,sustainability_cert,room_type,double_bed,single_bed,free_cancellation,breakfast_included,pay_at_hotel,location_score,free_taxi,week,type,destination
0,Hotel Meslay Republique,https://www.booking.com/hotel/fr/meslay-republ...,€,1143,1.4,8.1,3153.0,Yes,No,Habitación Doble Estándar - 2 camas,No,Yes,No,No,No,8.2,No,1,solitary,Paris
1,Superbe Triplex au Coeur de Montorgueil,https://www.booking.com/hotel/fr/superbe-tripl...,€,1272,1.1,8.5,28.0,Yes,No,Casa con 1 dormitorio,Yes,No,No,No,No,8.4,No,1,solitary,Paris
2,Moxy Paris Bastille,https://www.booking.com/hotel/fr/moxy-paris-ba...,€,1311,1.4,8.0,1337.0,Yes,No,MOXY Sleeper con cama grande,Yes,No,Yes,Yes,Yes,8.2,No,1,solitary,Paris
3,MBA Splendide Appart Saint-Michel Paris Centre,https://www.booking.com/hotel/fr/splendide-app...,€,776,0.8,7.1,15.0,Yes,No,Apartamento - Planta baja,No,No,No,No,No,NaN,No,1,solitary,Paris
4,Paris Bastille,https://www.booking.com/hotel/fr/hotel-paris-b...,€,1304,1.5,8.2,1919.0,Yes,No,Habitación Doble - 1 o 2 camas,Yes,Yes,No,No,No,8.6,No,1,solitary,Paris
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45,Ground Floor with private garden Bellresguard,https://www.booking.com/hotel/es/ground-floor-...,€,993,NaN,9.2,45.0,Yes,No,Apartamento con vistas al jardín,Yes,Yes,Yes,Yes,No,9.4,No,3,friends,Havana
46,Villa Marcos Lago Esperanza,https://www.booking.com/hotel/es/villa-lago-es...,€,947,NaN,9.0,31.0,Yes,No,Casa,Yes,Yes,No,No,No,9.0,No,3,friends,Havana
47,"Can Rafelino, pool and relax",https://www.booking.com/hotel/es/can-rafelino-...,€,1270,NaN,9.0,3.0,Yes,No,Villa con piscina privada,Yes,Yes,No,No,No,10.0,No,3,friends,Havana
48,Can Content - Playa de Muro,https://www.booking.com/hotel/es/can-content-m...,€,1211,NaN,9.3,6.0,Yes,No,Casa Unifamiliar de 3 dormitorios,Yes,Yes,Yes,Yes,No,10.0,No,3,friends,Havana


In [19]:
accommodations.to_csv("../data/accommodations/acommodations_cleaned.csv")

## 1.3 Activities

In [20]:
activities_df = pd.read_csv("../data/activities/civitatis_activities.csv")

In [21]:
activities_df.info()
activities_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1972 entries, 0 to 1971
Data columns (total 17 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Unnamed: 0       1972 non-null   int64  
 1   activity_name    1940 non-null   object 
 2   description      1972 non-null   object 
 3   url              1940 non-null   object 
 4   image            1972 non-null   object 
 5   image2           1632 non-null   object 
 6   available_days   1972 non-null   object 
 7   available_times  1972 non-null   object 
 8   duration         1972 non-null   object 
 9   latitude         1940 non-null   float64
 10  longitude        1940 non-null   float64
 11  address          1277 non-null   object 
 12  price            1866 non-null   float64
 13  currency         1866 non-null   object 
 14  category         1729 non-null   object 
 15  week             1972 non-null   int64  
 16  destination      1972 non-null   object 
dtypes: float64(3),

,Unnamed: 0,activity_name,description,url,image,image2,available_days,available_times,duration,latitude,longitude,address,price,currency,category,week,destination
0,0,Tour privado por Roma ¡Tú eliges!,Si queréis conocer la Ciudad del Amor de forma...,www.civitatis.com/es/roma/tour-privado-roma/,www.civitatis.com/f/italia/roma/tour-privado-r...,NaN,"['05', '08', '09', '06', '10', '07']","[['9:00', '10:00', '11:00', '12:00', '13:00', ...",2 horas,41.906658,12.481990,"19, Piazza di Spagna, Campo Marzio, Municipio ...",18.2,EUR,Español,1,roma
1,1,Visita guiada por la Basílica de San Pedro,Historia y arte se entremezclan en esta visita...,www.civitatis.com/es/roma/subida-cupula-san-pe...,"www.civitatis.comdata:image/gif;base64,R0lGODl...",www.civitatis.com/f/italia/roma/subida-cupula-...,"['09', '07', '05', '08']","[['8:15'], ['8:15'], ['8:15'], ['8:15']]",1h 30m -3h,41.907179,12.456438,"Minimarket, 27, Via Vespasiano, Prati, Municip...",10.5,EUR,Español,1,roma
2,2,Excursión a Venecia en tren de alta velocidad,En esta excursión a Venecia en tren de alta ve...,www.civitatis.com/es/roma/excursion-venecia/,"www.civitatis.comdata:image/gif;base64,R0lGODl...",www.civitatis.com/f/italia/roma/excursion-vene...,"['09', '07']","[['7:30'], ['7:30']]",15 horas,41.901980,12.503145,"48, Via Marsala, Castro Pretorio, Municipio Ro...",54.6,EUR,Español,1,roma
3,3,Visita guiada por los Museos Capitolinos y alr...,El Monumento a Víctor Manuel II y la Plaza del...,www.civitatis.com/es/roma/visita-guiada-museos...,"www.civitatis.comdata:image/gif;base64,R0lGODl...",www.civitatis.com/f/italia/roma/visita-guiada-...,"['10', '08', '06']","[['9:30', '16:30'], ['9:30', '16:30'], ['9:30'...",3 horas,41.893293,12.482938,"Piazza del Campidoglio, Campitelli, Municipio ...",14.0,EUR,Español,1,roma
4,4,Tour por las plazas y fuentes de Roma,"La Fontana de Trevi, la Piazza Navona o el Cam...",www.civitatis.com/es/roma/paseo-plazas-fuentes...,"www.civitatis.comdata:image/gif;base64,R0lGODl...",www.civitatis.com/f/italia/roma/paseo-plazas-f...,"['09', '07']","[['15:00'], ['15:00']]",2h 30m,41.903663,12.488483,"Fontana del Tritone, Piazza Barberini, Trevi, ...",10.0,EUR,Español,1,roma


Here, in contrast to the 2 previous groups of dataframes, the activities dataframes need some re-formatting.

In [22]:
activities_dataframes = [activities_df]

for activity_dataframe in activities_dataframes:
    activity_dataframe["image"] = np.where(~activity_dataframe["image2"].isna(), activity_dataframe["image2"],activity_dataframe["image"])

    # make lists literal, as they are str
    activity_dataframe['available_days'] = activity_dataframe['available_days'].apply(ast.literal_eval)
    activity_dataframe['available_times'] = activity_dataframe['available_times'].apply(ast.literal_eval)

    # add a 'd' to account for days when there's a hypen followe by a digit and a 'd'
    activity_dataframe["duration"] = np.where(
    activity_dataframe["duration"].str.contains(r" -\d"), 
    activity_dataframe["duration"].str.replace(r" -(\d)", r"d-\1", regex=True),
    activity_dataframe["duration"]
)

    # split duration into duration min and max
    activity_dataframe[["duration_min","duration_max"]] = activity_dataframe["duration"].str.split("-",expand=True)
    activity_dataframe[["duration_min","duration_max"]]
    activity_dataframe['duration_max'].fillna("0",inplace=True)

    # format irregular durations as hours
    activity_dataframe['hours_min'] = activity_dataframe['duration_min'].apply(convert_to_hours)
    activity_dataframe['hours_max'] = activity_dataframe['duration_max'].apply(convert_to_hours)

    # make min and max equal if no max
    activity_dataframe['hours_max'] = np.where(activity_dataframe['hours_max']==0 | activity_dataframe['hours_max'].isna(), activity_dataframe['hours_min'],activity_dataframe['hours_max'])

    #drop columns and display head
    activity_dataframe.drop(columns=["image2","duration","duration_min","duration_max"],inplace=True)
    print("\n\n\n")
    activity_dataframe.info()

    display(activity_dataframe.head())





<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1972 entries, 0 to 1971
Data columns (total 17 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Unnamed: 0       1972 non-null   int64  
 1   activity_name    1940 non-null   object 
 2   description      1972 non-null   object 
 3   url              1940 non-null   object 
 4   image            1972 non-null   object 
 5   available_days   1972 non-null   object 
 6   available_times  1972 non-null   object 
 7   latitude         1940 non-null   float64
 8   longitude        1940 non-null   float64
 9   address          1277 non-null   object 
 10  price            1866 non-null   float64
 11  currency         1866 non-null   object 
 12  category         1729 non-null   object 
 13  week             1972 non-null   int64  
 14  destination      1972 non-null   object 
 15  hours_min        1675 non-null   float64
 16  hours_max        1675 non-null   float64
dtypes: float64

C:\Users\Miguel\AppData\Local\Temp\ipykernel_36624\3760787029.py:20: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  activity_dataframe['duration_max'].fillna("0",inplace=True)


,Unnamed: 0,activity_name,description,url,image,available_days,available_times,latitude,longitude,address,price,currency,category,week,destination,hours_min,hours_max
0,0,Tour privado por Roma ¡Tú eliges!,Si queréis conocer la Ciudad del Amor de forma...,www.civitatis.com/es/roma/tour-privado-roma/,www.civitatis.com/f/italia/roma/tour-privado-r...,"[05, 08, 09, 06, 10, 07]","[[9:00, 10:00, 11:00, 12:00, 13:00, 14:00, 15:...",41.906658,12.481990,"19, Piazza di Spagna, Campo Marzio, Municipio ...",18.2,EUR,Español,1,roma,2.0,2.0
1,1,Visita guiada por la Basílica de San Pedro,Historia y arte se entremezclan en esta visita...,www.civitatis.com/es/roma/subida-cupula-san-pe...,www.civitatis.com/f/italia/roma/subida-cupula-...,"[09, 07, 05, 08]","[[8:15], [8:15], [8:15], [8:15]]",41.907179,12.456438,"Minimarket, 27, Via Vespasiano, Prati, Municip...",10.5,EUR,Español,1,roma,721.0,3.0
2,2,Excursión a Venecia en tren de alta velocidad,En esta excursión a Venecia en tren de alta ve...,www.civitatis.com/es/roma/excursion-venecia/,www.civitatis.com/f/italia/roma/excursion-vene...,"[09, 07]","[[7:30], [7:30]]",41.901980,12.503145,"48, Via Marsala, Castro Pretorio, Municipio Ro...",54.6,EUR,Español,1,roma,15.0,15.0
3,3,Visita guiada por los Museos Capitolinos y alr...,El Monumento a Víctor Manuel II y la Plaza del...,www.civitatis.com/es/roma/visita-guiada-museos...,www.civitatis.com/f/italia/roma/visita-guiada-...,"[10, 08, 06]","[[9:30, 16:30], [9:30, 16:30], [9:30, 16:30]]",41.893293,12.482938,"Piazza del Campidoglio, Campitelli, Municipio ...",14.0,EUR,Español,1,roma,3.0,3.0
4,4,Tour por las plazas y fuentes de Roma,"La Fontana de Trevi, la Piazza Navona o el Cam...",www.civitatis.com/es/roma/paseo-plazas-fuentes...,www.civitatis.com/f/italia/roma/paseo-plazas-f...,"[09, 07]","[[15:00], [15:00]]",41.903663,12.488483,"Fontana del Tritone, Piazza Barberini, Trevi, ...",10.0,EUR,Español,1,roma,2.5,2.5


Data values are now correct, as well as data types and can be used for analysis. Some data like the available days and available times, will have to be exploded in order to be usable for analysis.

# 2. Saving clean data

## 2.1 Flights

In [23]:
flight_itinieraries.to_csv("../data/flights/flight_itinieraries_cleaned.csv")

## 2.2 Accommodations

In [24]:
accommodations.to_csv("../data/accommodations/acommodations_cleaned.csv")

## 2.3 Activities

In [25]:
activities_df.to_csv("../data/activities/civitatis_activities_cleaned.csv")